In [ ]:
!pip install pyspark

     |████████████████████████████████| 204.8MB 71kB/s 
     |████████████████████████████████| 204kB 19.3MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.2-py2.py3-none-any.whl size=205186687 sha256=c24b73293eedfe9f7cca68ab105a6ded25c004e3b332e851a555517d00d5239f
  Stored in directory: /root/.cache/pip/wheels/8b/09/da/c1f2859bcc86375dc972c5b6af4881b3603269bcc4c9be5d16
Successfully built pyspark


# Thuật toán Apriori trên PySpark

## Import

In [ ]:
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from google.colab import drive
from pyspark.sql import SQLContext
drive.mount('/content/drive',force_remount=True)
import collections
conf = SparkConf().setMaster("local").setAppName("AprioriAlgorithm")
sc = SparkContext.getOrCreate(conf=conf)

Mounted at /content/drive


## Các hàm hổ trợ thuật toán

In [ ]:
# tạo ra các cặp candidate tiếp theo: C(k) =  tổ hợp chập k của n phần tử L(k-1)
def create_C_k(l_k, k):
  next_c = [var1 | var2 for index, var1 in enumerate(l_k) for var2 in l_k[index + 1:] if list(var1)[:k - 2] == list(var2)[:k - 2]]
  return next_c

# Đưa ra support thoả điều kiện lớn hơn hoặc bằng min_support
def get_support_valid(x, broadcastVar):
  x_sup = len([1 for t in broadcastVar.value if x.issubset(t)])
  if x_sup >= min_sup:
      return x, x_sup
  else:
      return ()

# tạo tập data L_k sau khi lọc các tập data phù hợp min_support
def create_L_k(sc, c_k, broadcastVar, min_sup):
  l_k = sc.parallelize(c_k).map(lambda x : get_support_valid(x, broadcastVar)).filter(lambda x:x).collect()
  return l_k

# Tính confidence
def calculateConfidence(item):
  # Parent item list
  parent = set(item[0][0])
  # Child item list
  # xét điều kiện child item có phải là string
  if(isinstance(item[1][0] , str)):
      child  = set([item[1][0]])
  else:
      child  = set(item[1][0])
  # Parent and Child support values
  parentSupport = item[0][1]
  childSupport = item[1][1]
  # Finds the item set confidence is going to be found
  support = (parentSupport / childSupport)*100
  return list([list(child) ,  list(parent.difference(child)) , support])

def filterForConf(item , total):      
  if(len(item[0][0]) > len(item[1][0])):
    if(checkItemSets(item[0][0] , item[1][0]) == False):
      pass
    else:
      return (item)       
  else:
    pass

# kiểm tra itemlist L_k -> lấy ra được l_k cuối cùng
def checkItemSets(item_1 , item_2):
  if(len(item_1) > len(item_2)):
    return all(any(k == l for k in item_1 ) for l in item_2)
  else:
    return all(any(k == l for k in item_2 ) for l in item_1)

## Đọc file store_data.csv


In [ ]:
text_file = sc.textFile("drive/MyDrive/BIGDATA/Week3/Apriori/store_data.csv")
# Print in the first of the file
print(text_file.first())

shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil


In [ ]:
# Split
itemset = text_file.map(lambda line: sorted([item for item in line.strip().split(',')]))
print(itemset.collect())

[['almonds', 'antioxydant juice', 'avocado', 'cottage cheese', 'energy drink', 'frozen smoothie', 'green grapes', 'green tea', 'honey', 'low fat yogurt', 'mineral water', 'olive oil', 'salad', 'salmon', 'shrimp', 'spinach', 'tomato juice', 'vegetables mix', 'whole weat flour', 'yams'], ['burgers', 'eggs', 'meatballs'], ['chutney'], ['avocado', 'turkey'], ['energy bar', 'green tea', 'milk', 'mineral water', 'whole wheat rice'], ['low fat yogurt'], ['french fries', 'whole wheat pasta'], ['light cream', 'shallot', 'soup'], ['frozen vegetables', 'green tea', 'spaghetti'], ['french fries'], ['eggs', 'pet food'], ['cookies'], ['burgers', 'cooking oil', 'eggs', 'mineral water', 'turkey'], ['champagne', 'cookies', 'spaghetti'], ['mineral water', 'salmon'], ['mineral water'], ['chicken', 'chocolate', 'cooking oil', 'honey', 'low fat yogurt', 'oil', 'shrimp'], ['eggs', 'turkey'], ['black tea', 'chicken', 'eggs', 'extra dark chocolate', 'fresh tuna', 'mineral water', 'salmon', 'spaghetti', 'tomat

In [ ]:
# Broadcast variables: biến broadcast lưu dữ liệu dưới dạng read-only cached và deserialized trên mỗi cluster -> giảm chi phí "giao tiếp"
broadcastVar = sc.broadcast(itemset.map(lambda x: set(x)).collect())
print(broadcastVar.value)

[{'mineral water', 'tomato juice', 'cottage cheese', 'spinach', 'frozen smoothie', 'vegetables mix', 'avocado', 'low fat yogurt', 'salad', 'yams', 'antioxydant juice', 'almonds', 'whole weat flour', 'olive oil', 'honey', 'energy drink', 'shrimp', 'green grapes', 'salmon', 'green tea'}, {'eggs', 'burgers', 'meatballs'}, {'chutney'}, {'turkey', 'avocado'}, {'mineral water', 'green tea', 'energy bar', 'milk', 'whole wheat rice'}, {'low fat yogurt'}, {'whole wheat pasta', 'french fries'}, {'soup', 'light cream', 'shallot'}, {'green tea', 'spaghetti', 'frozen vegetables'}, {'french fries'}, {'eggs', 'pet food'}, {'cookies'}, {'burgers', 'eggs', 'turkey', 'mineral water', 'cooking oil'}, {'spaghetti', 'cookies', 'champagne'}, {'mineral water', 'salmon'}, {'mineral water'}, {'low fat yogurt', 'chocolate', 'honey', 'cooking oil', 'shrimp', 'chicken', 'oil'}, {'eggs', 'turkey'}, {'salmon', 'eggs', 'turkey', 'mineral water', 'spaghetti', 'fresh tuna', 'black tea', 'chicken', 'extra dark chocolat

In [ ]:
k = 1
# candidate -> c(1) với k = 1
# flatmap -> flatMap([[1,2], [3,4]]) = [1, 2, 3, 4]
c_k = itemset.flatMap(lambda x: x).distinct().collect()
# chuyển về set() để sử dụng các toán tử
c_k = [{x} for x in c_k]
print("Candidate 1:",len(c_k))
print(c_k)

Candidate 1: 120
[{'almonds'}, {'antioxydant juice'}, {'avocado'}, {'cottage cheese'}, {'energy drink'}, {'frozen smoothie'}, {'green grapes'}, {'green tea'}, {'honey'}, {'low fat yogurt'}, {'mineral water'}, {'olive oil'}, {'salad'}, {'salmon'}, {'shrimp'}, {'spinach'}, {'tomato juice'}, {'vegetables mix'}, {'whole weat flour'}, {'yams'}, {'burgers'}, {'eggs'}, {'meatballs'}, {'chutney'}, {'turkey'}, {'energy bar'}, {'milk'}, {'whole wheat rice'}, {'french fries'}, {'whole wheat pasta'}, {'light cream'}, {'shallot'}, {'soup'}, {'frozen vegetables'}, {'spaghetti'}, {'pet food'}, {'cookies'}, {'cooking oil'}, {'champagne'}, {'chicken'}, {'chocolate'}, {'oil'}, {'black tea'}, {'extra dark chocolate'}, {'fresh tuna'}, {'tomatoes'}, {'protein bar'}, {'pasta'}, {'pepper'}, {'red wine'}, {'shampoo'}, {'rice'}, {'sparkling water'}, {'body spray'}, {'ham'}, {'pancakes'}, {'grated cheese'}, {'toothpaste'}, {'white wine'}, {'fresh bread'}, {'parmesan cheese'}, {'escalope'}, {'ground beef'}, {'he

## Chạy thuật toán: min_sup = 150 - min_confidence = 40%

In [ ]:
min_sup = 150
min_Confidence = 0.4
All_L_itemsets = list()
while len(c_k) > 0:
  print("=====================================================")
  # create l_k
  print("C{}: {}".format(k, c_k))
  l_k = create_L_k(sc, c_k, broadcastVar, min_sup)
  print("L{}: {}".format(k, l_k))

  # x_sup = len([1 for t in broadcastVar.value if {"milk"}.issubset(t)]) # 972

  # filter: lọc các phần tử không phù hợp
  # l_k_test = sc.parallelize(c_k).map(lambda x: len([1 for t in broadcastVar.value if x.issubset(t)])).collect()
  # l_k_test_1 = sc.parallelize(c_k).map(lambda x: len([1 for t in broadcastVar.value if x.issubset(t)])).filter(lambda x : x).collect()
  # test = [1 for t in broadcastVar.value]

  # print(l_k_test)
  # print(l_k_test_1)

  # print([set(item) for item in map(lambda x: x[0], l_k)])

  print("Số lượng L{}: {}".format(k, len(l_k)))
  if len(l_k) != 0:
    for i in l_k:
      # chuyển kiểu dữ liệu -> áp data vào cartesian
      # print((list(x for x in i[0]),i[1]))
      All_L_itemsets.append((list(x for x in i[0]),i[1]))

  # tạo candidate(k+1)
  k += 1
  c_k = create_C_k([set(item) for item in map(lambda x: x[0], l_k)], k)
  
print("=====================================================")
# các cặp có min_sup phù hợp với từng C(k)
rdd = sc.parallelize(All_L_itemsets, numSlices=1)
print(rdd.collect())

C1: [{'almonds'}, {'antioxydant juice'}, {'avocado'}, {'cottage cheese'}, {'energy drink'}, {'frozen smoothie'}, {'green grapes'}, {'green tea'}, {'honey'}, {'low fat yogurt'}, {'mineral water'}, {'olive oil'}, {'salad'}, {'salmon'}, {'shrimp'}, {'spinach'}, {'tomato juice'}, {'vegetables mix'}, {'whole weat flour'}, {'yams'}, {'burgers'}, {'eggs'}, {'meatballs'}, {'chutney'}, {'turkey'}, {'energy bar'}, {'milk'}, {'whole wheat rice'}, {'french fries'}, {'whole wheat pasta'}, {'light cream'}, {'shallot'}, {'soup'}, {'frozen vegetables'}, {'spaghetti'}, {'pet food'}, {'cookies'}, {'cooking oil'}, {'champagne'}, {'chicken'}, {'chocolate'}, {'oil'}, {'black tea'}, {'extra dark chocolate'}, {'fresh tuna'}, {'tomatoes'}, {'protein bar'}, {'pasta'}, {'pepper'}, {'red wine'}, {'shampoo'}, {'rice'}, {'sparkling water'}, {'body spray'}, {'ham'}, {'pancakes'}, {'grated cheese'}, {'toothpaste'}, {'white wine'}, {'fresh bread'}, {'parmesan cheese'}, {'escalope'}, {'ground beef'}, {'herb & pepper'}

## Tính confidence và tìm các association rule



In [ ]:
# cartesian: Khi gọi 1 tập dữ liệu kiểu T và U, nó sẽ trả về tập dữ liệu mới (T,U)
cacl = rdd.cartesian(rdd)
print(cacl.collect())
print("----------------------")
transition = cacl.count()

# Gán điều kiện filter: true false
baseRddConfidence = cacl.filter(lambda item: filterForConf(item , transition))
# test
print("filter thành parent itemlist và child itemlist (loại bỏ các L trước dùng hàm checkItemSets()):")
print(baseRddConfidence.collect())
print("-------------------------------")
baseRddConfidence = baseRddConfidence.map(lambda item: calculateConfidence(item))
# before -> after : conf
print(baseRddConfidence.collect())

[((['almonds'], 153), (['almonds'], 153)), ((['almonds'], 153), (['avocado'], 250)), ((['almonds'], 153), (['cottage cheese'], 239)), ((['almonds'], 153), (['energy drink'], 200)), ((['almonds'], 153), (['frozen smoothie'], 475)), ((['almonds'], 153), (['green tea'], 991)), ((['almonds'], 153), (['honey'], 356)), ((['almonds'], 153), (['low fat yogurt'], 574)), ((['almonds'], 153), (['mineral water'], 1788)), ((['almonds'], 153), (['olive oil'], 494)), ((['almonds'], 153), (['salmon'], 319)), ((['almonds'], 153), (['shrimp'], 536)), ((['almonds'], 153), (['tomato juice'], 228)), ((['almonds'], 153), (['vegetables mix'], 193)), ((['almonds'], 153), (['burgers'], 654)), ((['almonds'], 153), (['eggs'], 1348)), ((['almonds'], 153), (['meatballs'], 157)), ((['almonds'], 153), (['turkey'], 469)), ((['almonds'], 153), (['energy bar'], 203)), ((['almonds'], 153), (['milk'], 972)), ((['almonds'], 153), (['whole wheat rice'], 439)), ((['almonds'], 153), (['french fries'], 1282)), ((['almonds'], 

In [ ]:
data = baseRddConfidence.collect()
## tạo một dataframe pySpark
sqlContext = SQLContext(sc)
columns = ["Before", "After" , "Confidence (%)"]
confidenceTable = sqlContext.createDataFrame(data = data , schema = columns)

## show dataframe
print("Các association rule của tập L thoả mãn min_support:")
print(confidenceTable.show())

Các association rule của tập L thoả mãn min_support:
+-----------------+-----------------+------------------+
|           Before|            After|    Confidence (%)|
+-----------------+-----------------+------------------+
|[frozen smoothie]|  [mineral water]|              32.0|
|  [mineral water]|[frozen smoothie]| 8.501118568232663|
|      [green tea]|  [mineral water]|23.511604439959637|
|  [mineral water]|      [green tea]|13.031319910514542|
|      [green tea]|           [eggs]| 19.27346115035318|
|           [eggs]|      [green tea]|14.169139465875372|
|      [green tea]|   [french fries]|21.594349142280524|
|   [french fries]|      [green tea]| 16.69266770670827|
|      [green tea]|      [spaghetti]|20.080726538849646|
|      [spaghetti]|      [green tea]|15.237366003062789|
|      [green tea]|      [chocolate]|17.759838546922303|
|      [chocolate]|      [green tea]|14.320585842148088|
| [low fat yogurt]|  [mineral water]|  31.3588850174216|
|  [mineral water]| [low fat yogurt

## Chọn association rule phù hợp với min_confidence = 40%

In [ ]:
print('Các Association rule thoả mãn điều kiện (confidence = 40%):')
result = []
# lọc các association rule có conf lớn hơn hoặc bằng min_confidence
for i in data:
  if float(i[-1]) >= float(min_Confidence*100):
    result.append(i)
confidenceValid = sqlContext.createDataFrame(data = result , schema = columns)

print(confidenceValid.show())

Các Association rule thoả mãn điều kiện (confidence = 40%):
+-------------+---------------+------------------+
|       Before|          After|    Confidence (%)|
+-------------+---------------+------------------+
|  [olive oil]|[mineral water]|41.902834008097166|
|       [soup]|[mineral water]| 45.64643799472295|
|[ground beef]|[mineral water]| 41.65535956580732|
+-------------+---------------+------------------+

None
